In [ ]:
import os
import dotenv
from neo4j import GraphDatabase

dotenv.load_dotenv()

True

## Connect to DB

In [7]:
# URI examples: "neo4j://localhost", "neo4j+s://xxx.databases.neo4j.io"
URI = os.getenv("NEO4J_URI")
AUTH = (os.getenv("NEO4J_USERNAME"), os.getenv("NEO4J_PASSWORD"))
DATABASE_NAME = os.getenv("NEO4J_DATABASE")

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()

## Create two Person nodes and a KNOWS relationship between them

In [9]:
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    summary = driver.execute_query("""
        CREATE (a:Person {name: $name})
        CREATE (b:Person {name: $friendName})
        CREATE (a)-[:KNOWS]->(b)
        """,
        name="Alice", friendName="David",
        database_=DATABASE_NAME,
    ).summary
    print("Created {nodes_created} nodes in {time} ms.".format(
        nodes_created=summary.counters.nodes_created,
        time=summary.result_available_after
    ))

Created 2 nodes in 28 ms.


## Retrieve all Person nodes who know other persons

In [10]:
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    records, summary, keys = driver.execute_query("""
        MATCH (p:Person)-[:KNOWS]->(:Person)
        RETURN p.name AS name
        """,
        database_=DATABASE_NAME,
    )

    # Loop through results and do something with them
    for record in records:
        print(record.data())  # obtain record as dict

    # Summary information
    print("The query `{query}` returned {records_count} records in {time} ms.".format(
        query=summary.query, records_count=len(records),
        time=summary.result_available_after
    ))

{'name': 'Alice'}
{'name': 'Alice'}
The query `
        MATCH (p:Person)-[:KNOWS]->(:Person)
        RETURN p.name AS name
        ` returned 2 records in 48 ms.


## Delete the Nodes

In [12]:
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    # This does not delete _only_ p, but also all its relationships!
    records, summary, keys = driver.execute_query("""
        MATCH (p:Person {name: $name})
        DETACH DELETE p
        """, name="Alice",
        database_=DATABASE_NAME,
    )
    print(f"Query counters: {summary.counters}.")

Query counters: {'_contains_updates': True, 'nodes_deleted': 2, 'relationships_deleted': 2}.
